from Bartholomew & Kwakkel (2020):


2.2. Multi-scenario many-objective robust decision making
Multi-scenario MORDM (Watson and Kasprzyk, 2017) is a further
refinement of MORDM. The main contribution of MORDM was the
use of a MOEA for finding a set of promising candidate solutions
which together capture the key trade-offs amongst competing objectives. However, this search uses a single reference scenario, and it is
unlikely that solutions that are optimal in a given scenario are also
optimally robust. Multi-scenario MORDM (Fig. 1(c)) addresses this by
performing a search for candidate strategies for several different reference scenarios. The additional scenarios for which search is performed
are selected from regions in the deep uncertainty space where candidate
solutions found for the first reference scenario are failing to meet their
objectives. So, one performs the four MORDM steps, and based on
the insights from scenario discovery, additional scenarios are selected
for which search is also performed. The goal of this is to build a
more diverse set of policy alternatives which are Pareto optimal under
different scenarios.
The selection of scenarios after the first MORDM iteration is a critical step in multi-scenario MORDM (Eker and Kwakkel, 2018). Watson
and Kasprzyk (2017) suggest picking scenarios based on the scenario
discovery results. The number of scenarios to select is left to the
analyst. Clearly, if the number of scenarios for which a search is
conducted increases, the chance of finding solutions that are robust
during the re-evaluation also increases. However, this comes at a substantial computational cost. To assist in balancing comprehensiveness
and computational cost, while making scenario selection transparent
and reproducible, Eker and Kwakkel (2018) present an approach for
finding the most policy relevant and maximally diverse set of scenarios.
Policy relevance is defined as scenarios that lead to poor outcomes and
the diversity criterion is based on Carlsen et al. (2016).

# A. Many-Objective Robust Decision Making with Multi-Scenario Search

In [3]:
# import libraries

from ema_workbench import (
    Model,
    MultiprocessingEvaluator,
    ScalarOutcome,
    IntegerParameter,
    optimize,
    Scenario, 
    save_results,
    load_results
)
from ema_workbench.em_framework import ArchiveLogger
from ema_workbench.em_framework.optimization import EpsilonProgress
from ema_workbench.util import ema_logging

from problem_formulation import get_model_for_problem_formulation

import matplotlib.pyplot as plt
import seaborn as sns

### A.1. Problem formulation

In [2]:
if __name__ == "__main__":
    ema_logging.log_to_stderr(ema_logging.INFO)

    model, steps = get_model_for_problem_formulation(3)

    reference_values = {
        "Bmax": 175,
        "Brate": 1.5,
        "pfail": 0.5,
        "discount rate 0": 3.5,
        "discount rate 1": 3.5,
        "discount rate 2": 3.5,
        "ID flood wave shape": 4,
    }
    scen1 = {}

    for key in model.uncertainties:
        name_split = key.name.split("_")

        if len(name_split) == 1:
            scen1.update({key.name: reference_values[key.name]})

        else:
            scen1.update({key.name: reference_values[name_split[1]]})

    ref_scenario = Scenario("reference", **scen1)

    # determine epsilon
    epsilon = [1e3] * len(model.outcomes)
    # convergence metrics
    convergence_metrics = [ArchiveLogger(
        "./archives",
        [l.name for l in model.levers],
        [o.name for o in model.outcomes],
        base_filename="multiobj_problem3_results.tar.gz",
    ),
        EpsilonProgress(),
    ]

    # number of functions evaluations
    nfe = 2  

    # multiprocessing
    with MultiprocessingEvaluator(model) as evaluator:
        results, convergence = evaluator.optimize(
            nfe=nfe,
            searchover="levers",
            epsilons=epsilon,
            convergence=convergence_metrics,
            reference=ref_scenario,
        )

KeyboardInterrupt: 

### A.2. Scenario selection

It is desirable to search for candidate solutions under a variety of scenarios, so that robustness of potential solutions is increased (Eker and Kwakkel, 2018).

In this study, our purpose for scenario selection is to find scenarios that potentially lead to solutions that are more robust when
the search phase of MORDM is conducted for each of these scenarios. Therefore, among the possible scenario selection criteria
listed above, we consider policy relevance and diversity as the
relevant ones. Policy-relevance is a problem-specific concept that
should reflect the decision-making concerns and preferences. In
this study, we choose to define policy-relevance with respect to
undesirable scenario conditions specified by the median values of
the scenario space. Dividing the scenario space at the median
values focuses the diversity analysis on a smaller number of policyrelevant scenarios regardless of their distribution. However, we do
not propose this as a general way to define policy relevance. For the
diversity criterion, we propose to use the specific diversity
maximization method introduced by Carlsen et al. (2016b).

Following Eker and Kwakkel (2018), scenario selection is conducted in three steps, namely
- 1. Scenario generation;
- 2. Filtering of policy-relevant scenarios;
- 3. Selecting maximally diverse scenarios.

#### A.2.1. Scenario generation

In this step, N scenarios will be generated using randomly sampled values of deep uncertainties and decision levers (Eker and Kwakkel, 2018) .

In [4]:
scenarios, outcomes = load_results("./results/potentialscenarios.tar.gz")

[MainProcess/INFO] results loaded successfully from C:\Users\nelen\EPA141A\final assignment\results\potentialscenarios.tar.gz


In [5]:
scenarios

,A.0_ID flood wave shape,A.1_Bmax,A.1_Brate,A.1_pfail,A.2_Bmax,A.2_Brate,A.2_pfail,A.3_Bmax,A.3_Brate,A.3_pfail,...,A.3_DikeIncrease 2,A.4_DikeIncrease 0,A.4_DikeIncrease 1,A.4_DikeIncrease 2,A.5_DikeIncrease 0,A.5_DikeIncrease 1,A.5_DikeIncrease 2,scenario,policy,model
0,66,220.340347,1.5,0.277751,317.625580,1.0,0.246968,114.478228,10.0,0.552078,...,0,0,0,0,0,0,0,23,Policy 0,dikesnet
1,39,287.676115,1.0,0.093811,246.091351,1.5,0.271764,101.893796,1.0,0.429600,...,0,0,0,0,0,0,0,40,Policy 0,dikesnet
2,50,106.756807,10.0,0.345841,333.340514,1.5,0.745971,82.951536,1.0,0.469837,...,0,0,0,0,0,0,0,65,Policy 0,dikesnet
3,76,56.356797,1.5,0.189130,196.888090,10.0,0.057171,280.790547,1.5,0.470550,...,0,0,0,0,0,0,0,125,Policy 0,dikesnet
4,132,298.587312,1.5,0.000580,156.903324,1.5,0.970702,232.692672,10.0,0.575071,...,0,0,0,0,0,0,0,155,Policy 0,dikesnet
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1546,89,46.645355,1.0,0.751029,277.455225,1.5,0.606332,204.774335,10.0,0.012546,...,0,0,0,0,0,0,0,4966,Policy 0,dikesnet
1547,62,52.666623,1.0,0.866610,52.132103,10.0,0.671048,73.254758,10.0,0.217036,...,0,0,0,0,0,0,0,4967,Policy 0,dikesnet
1548,7,320.950831,1.5,0.533701,300.887555,1.0,0.992886,134.842915,1.0,0.166615,...,0,0,0,0,0,0,0,4973,Policy 0,dikesnet
1549,49,260.621182,1.5,0.756180,245.847333,1.5,0.588074,146.423040,1.0,0.140686,...,0,0,0,0,0,0,0,4976,Policy 0,dikesnet


#### A.2.2. Filtering of policy-relevant scenarios

Due to computational restrictions, only a limited amount of scenarios is used. In this step, scenarios are filtered based on their policy relevance.

In the next step, scenarios will be selected using an indicator for scenario diversity.


#### A.2.3. Selecting maximally diverse scenarios

It is desirable to select maximally diverse scenarios, so that ... ? In this step, maximally diverse scenarios will be selected following the approach of Carlsen et al. (2016).

#### Normalizing all outcomes

In [6]:
from sklearn import preprocessing
import pandas as pd

experiments_of_interest = scenarios['scenario']
outcomes_df = pd.DataFrame({k:v[experiments_of_interest] for k,v in outcomes.items()})
# normalize outcomes on unit interval to ensure equal weighting of outcomes
x = outcomes_df.values
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(x)
normalized_outcomes = pd.DataFrame(x_scaled, columns=outcomes_df.columns)

In [7]:
normalized_outcomes.describe()

,A.1 Total Costs,A.1_Expected Number of Deaths,A.2 Total Costs,A.2_Expected Number of Deaths,A.3 Total Costs,A.3_Expected Number of Deaths,A.4 Total Costs,A.4_Expected Number of Deaths,A.5 Total Costs,A.5_Expected Number of Deaths,RfR Total Costs,Expected Evacuation Costs
count,1551.000000,1551.000000,1551.000000,1551.000000,1551.000000,1551.000000,1551.000000,1551.000000,1551.000000,1551.000000,1551.0,1551.0
mean,0.147201,0.202552,0.281586,0.388328,0.340187,0.461868,0.051411,0.062236,0.026690,0.033302,0.0,0.0
std,0.263325,0.352418,0.274857,0.363970,0.338126,0.449401,0.146457,0.174582,0.106144,0.129834,0.0,0.0
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0
25%,0.000000,0.000000,0.026928,0.041821,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0
50%,0.018770,0.024395,0.206562,0.294506,0.242730,0.342232,0.000000,0.000000,0.000000,0.000000,0.0,0.0
75%,0.096507,0.144777,0.519384,0.662183,0.670450,0.964850,0.002960,0.003722,0.000000,0.000000,0.0,0.0
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.0,0.0


#### Creating all possible permutations of 4 scenarios

In [8]:

# # DO NOT RUN!!!!!!!!!!! MEMORY ERROR
#
# import itertools
#
# n_scen = experiments_of_interest.shape[0]
# print("Number of scenarios generated:", n_scen)
# indices = range(n_scen)
# set_size = 4
# combinations = itertools.combinations(indices, set_size)
# combinations = list(combinations)
# print("Number of combinations generated:", len(combinations))

Number of scenarios generated: 1551


MemoryError: 

#### Calculating diversity of each of the combinations

In [6]:
#The following code snippet is adopted from assignment 10 model answers (Kwakkel, 2024)

from concurrent.futures import ProcessPoolExecutor
import os
import functools
from scipy.spatial.distance import pdist, squareform
import random
# the relevant code is in a .py file to esnure parallization works within the notebook
from scenario_selection import find_maxdiverse_scenarios
import numpy as np

# calculate the pairwise distances between the normalized outcomes
distances = squareform(pdist(normalized_outcomes.values))

cores = os.cpu_count()
partial_function = functools.partial(find_maxdiverse_scenarios, distances)

In [ ]:
sampled_combinations = random.sample(combinations, 100000)

In [ ]:
# setup the pool of workers and split the calculations over the set of cores
with ProcessPoolExecutor(max_workers=cores) as executor:
    worker_data = np.array_split(sampled_combinations, cores)
    results = [e for e in executor.map(partial_function, worker_data)]
    results = list(itertools.chain.from_iterable(results))

#### Check most diverse results

In [ ]:
# #The following code snippet is adopted from assignment 10 model answers (Kwakkel, 2024)
#
# results.sort(key=lambda entry:entry[0], reverse=True)
# most_diverse = results[0]
# most_diverse

#### Create scenarios using most diverse scenarios

In [ ]:
# #The following code snippet is adopted from assignment 10 model answers (Kwakkel, 2024)
#
# from ema_workbench import Scenario
#
# selected = experiments.loc[most_diverse[1], ['b', 'delta', 'mean', 'q', 'stdev']]
# scenarios = [Scenario(f"{index}", **row) for index, row in selected.iterrows()]
#
# for scenario in scenarios:
#     print(scenario)

### A.3. Generating candidate solutions

Following the scenario selection of the previous step, in this step the Multi-Objective Evolutionary Algorithm (MOEA) epsilon-NSGAII algorithm (Kollat and Reed, 2006)  is used to generate candidate solutions, following the approach of Eker and Kwakkel (2018). + explain how epsilon-NSGAII algorithm works + explain benefits of epsilon-NSGAII algorithm for this case

#### Creating a function to optimize under a certain scenario

In [ ]:

#The following code snippet is adopted from assignment 10 model answers (Kwakkel, 2024)

# from ema_workbench import MultiprocessingEvaluator, ema_logging
# from ema_workbench.em_framework.evaluators import BaseEvaluator
#
# from ema_workbench.em_framework.optimization import (ArchiveLogger,
#                                                      EpsilonProgress,
#                                                      to_problem, epsilon_nondominated)
#
# ema_logging.log_to_stderr(ema_logging.INFO)
#
# def optimize(scenario, nfe, model, epsilons):
#     results = []
#     convergences = []
#     problem = to_problem(model, searchover="levers")
#
#     with MultiprocessingEvaluator(model) as evaluator:
#         for i in range(5):
#             convergence_metrics = [
#                 ArchiveLogger(
#                     "./archives",
#                     [l.name for l in model.levers],
#                     [o.name for o in model.outcomes],
#                     base_filename=f"assignemnt_9_{scenario.name}_seed_{i}.tar.gz",
#                 ),
#                 EpsilonProgress(),
#             ]
#
#             result, convergence = evaluator.optimize(nfe=nfe, searchover='levers',
#                                          convergence=convergence_metrics,
#                                          epsilons=epsilons,
#                                          reference=scenario)
#
#             results.append(result)
#             convergences.append(convergence)
#
#     # merge the results using a non-dominated sort
#     reference_set = epsilon_nondominated(results, epsilons, problem)
#
#     return reference_set, convergences
#
#
# results = []
# for scenario in scenarios:
#     epsilons = [0.05,]*len(model.outcomes)
#
#     # note that 100000 nfe is again rather low to ensure proper convergence
#     results.append(optimize(scenario, 1e5, model, epsilons))

#### Scenario 1 (reference scenario)

#### Scenario 2

#### Scenario 3

#### Scenario 4

#### Assess convergence

#### Combining the pareto set of solutions found for each scenario

#### (If you have a very large number of policies, you can choose to down sample your policies in some reasoned way (e.g., picking min and max on each objective, slicing across the pareto front with a particular step size). As a rule of thumb, try to limit the set of policies to at most 50.)

In [ ]:
# from ema_workbench import Policy
#
# policies = []
# for i, (result, _) in enumerate(results):
#     result = result.iloc[:, 0:5]
#     for j, row in result.iterrows():
#         policy = Policy(f'scenario {i} option {j}', **row.to_dict())
#         policies.append(policy)

#### Re-evaluate under deep uncertainty
Re-evaluate the combined set of solutions over 1000 scenarios sampled using LHS

In [ ]:
# with MultiprocessingEvaluator(model) as evaluator:
#     reeevaluation_results = evaluator.perform_experiments(1000, policies=policies)

### A.4. Tradeoff analysis

In [ ]:
# experiments, outcomes = reeevaluation_results
#....

**Signal-to-Noise**\
To quantify the robustness we can use multiple robustness metrices. The first one that we use is the **signal-to-noise ratio**. This is the mean of a dataset divided by the standard deviation. If we want to minimize the outcomes, a low mean and a low standard deviation is preferred.

In [ ]:
def signal_to_noise(data, direction):
    "Calculate the signal-to-noise ratio of a dataset with outcome directions (minimize or maximize)"
    mean = np.mean(data)
    std = np.std(data) 
    
    if direction==ScalarOutcome.MAXIMIZE:
        return mean/std
    else:
        return mean*std 

In [ ]:
# Empty dictionary to collect the ratio scores of every policy:
overall_scores = {}

# Loop over all policies and outcomes respectively:
for policy in np.unique(experiments['policy']):
    scores = {}
    
    logical = experiments['policy']==policy
    
    for outcome in model.outcomes:
        value  = outcomes[outcome.name][logical]
        sn_ratio = s_to_n(value, outcome.kind)
        scores[outcome.name] = sn_ratio
    overall_scores[policy] = scores
    
# Create dataframe for visualisation purposes:
scores = pd.DataFrame.from_dict(overall_scores).T
scores

In [ ]:
# Visualise tradeoffs on a parallel plot:
from ema_workbench.analysis import parcoords

data = scores
limits = parcoords.get_limits(data)
limits.loc[0, ['A.1 Total Costs', 'A.1_Expected Number of Deaths', 'A.2 Total Costs', 'A.2_Expected Number of Deaths', 'A.3 Total Costs', 'A.3_Expected Number of Deaths','RfR Total Costs', 'Expected Evacuation Costs']] = 0

paraxes = parcoords.ParallelAxes(limits)
paraxes.plot(data)
paraxes.invert_axis('A.1 Total Costs')
plt.show()

**Minimax regret**\
Another robustness metric that we use for our analysis is the **minimax regret** metric. This metric computes the regret for each option, then takes the ones with the maximum regret (worst-case), and then chooses the options that minimize this maximum regret. We define regret as the difference between the performance of a policy in a scenario and the performance of the best possible result in that scenario or the reference policy. The province of Gelderland is in favour of policy options with low maximum regret values, because the province is responsible for the safety of her region and pays the policies with government money. So, the province has a high level of risk aversion and therefore this robustness metric is suitable for our analysis.

Source: https://agupubs.onlinelibrary.wiley.com/doi/epdf/10.1002/2017EF000649

In [ ]:
# Empty dictionaries to get the overall and maximum regret:
overall_regret = {}
max_regret = {}

# Loop over all policies and outcomes respectively:
for outcome in model.outcomes:
    policy_column = experiments['policy']
    
    # Create DataFrame to store the outcome, name of the policy and scenario:
    data = pd.DataFrame({outcome.name: outcomes[outcome.name], 
                         "policy":experiments['policy'],
                         "scenario":experiments['scenario']})
    
    # Organise the data:
    data = data.pivot(index='scenario', columns='policy')
    
    # Flatten the hierarchical index from pivoting
    data.columns = data.columns.get_level_values(1)
    
    # Take the absolute difference of the maximum across the row and the actual values in the row
    outcome_regret = (data.max(axis=1).values[:, np.newaxis] - data).abs()
    
    overall_regret[outcome.name] = outcome_regret
    max_regret[outcome.name] = outcome_regret.max()

# Visualise results in a heatmap:
max_regret = pd.DataFrame(max_regret)
sns.heatmap(max_regret/max_regret.max(), cmap='viridis', annot=True)
plt.show()

In [ ]:
# Visualise tradeoffs on a parallel plot:
from ema_workbench.analysis import parcoords

colors = sns.color_palette()

data = max_regret
limits = parcoords.get_limits(data)
limits.loc[0, ['A.1 Total Costs', 'A.1_Expected Number of Deaths', 'A.2 Total Costs', 'A.2_Expected Number of Deaths', 'A.3 Total Costs', 'A.3_Expected Number of Deaths','RfR Total Costs', 'Expected Evacuation Costs']] = 0

paraxes = parcoords.ParallelAxes(limits)
for i, (index, row) in enumerate(data.iterrows()):
    paraxes.plot(row.to_frame().T, label=str(index), color=colors[i])
paraxes.legend()
plt.show()